In [1]:
import pandas as pd
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
import os, sys, re, torch, json, glob, argparse, gc, ast, pickle, requests
import numpy as np
from tqdm import tqdm
from ast import literal_eval
from sentence_transformers import SentenceTransformer, util
from tokenizers import AddedToken
from peft import PeftModel, PeftConfig
from scripts.formatting_results import *
from scripts.bert_filtering import *
from scripts.negation import *
from scripts.prompting import *
from scripts.utils import *
from scripts.llama_vision_engine import *
#from scripts.llava_med_engine import *
device = "cuda" if torch.cuda.is_available() else "cpu"
gc.collect()
torch.cuda.empty_cache()

/home/nguyenqm/miniconda3/envs/phenogpt2/lib/python3.11/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
# import random
# # Fix random seeds
# seed = 42
# os.environ["PYTHONHASHSEED"] = str(seed)
# os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
# torch.use_deterministic_algorithms(True)
# # End

In [3]:
## Provide the model directory here
#model_id = './models/PhenoGPT2'
model_id = '/home/nguyenqm/projects/github/PhenoGPT2/phenogpt2_EHR_L318B_text_FPFF/model/'
#model_id = '/home/nguyenqm/projects/github/PhenoGPT2/phenogpt2_qwen3_ehr_8b_ft_updated/model'
## Set True if your model is ft with LoRA, otherwise False
lora = False
## Provide the input dictionary file
#input_dir = './data/example/text_examples.json'
input_dir = '/home/nguyenqm/projects/MedicalDBs/Arcus/phenogpt2/arcus_notes_phenogpt2_updated.json'
data_input = read_input(input_dir)
## Replication (you can try to run three times for a sample then combine them but default to be 0)
i = 0
## Whether you want to remove negated findings (note that higher recall when negation = False)
negation = True
## If you don't want to split then keep 'wc' as 0; otherwise provide word size you want for each chunk
wc = 0
if wc != 0: 
    bert_tokenizer, bert_model = bert_init(local_dir = "./models/bert_filtering/")

In [4]:
# Determine processing mode
use_text = True
use_vision = False
vision_model = 'llama-vision'
###
# Vision model setup (only if vision is enabled)
print(f"use_vision: {use_vision}")
if use_vision:
    #base_ckpt = "/mnt/isilon/wang_lab/shared/LlaMA3.2-vision-instruct"
    phenogpt2_vision = LLaMA_Generator(os.getcwd() + "/models/llama-vision-phenogpt2/")#, base_ckpt)
#     vision_model = vision_model.lower() if vision_model else "llama-vision"
#     if vision_model == "llava-med":
#         phenogpt2_vision = LLaVA_Generator(os.path.join(os.getcwd(), "llava-med-phenogpt2"))
#     elif vision_model == "llama-vision":
#         phenogpt2_vision = LLaMA_Generator(os.path.join(os.getcwd(), "llama-vision-phenogpt2"))
#     else:
#         raise ValueError(f"Unsupported vision model '{vision_model}'. Use 'llava-med' or 'llama-vision'.")

use_vision: False


In [5]:
if lora:
    peft_config = PeftConfig.from_pretrained(model_id)
    # Get path to this file (inference.py)
    #current_file = os.path.abspath(__file__)

    # Get path to phenogpt2 root (go up 2 levels: scripts/ -> phenogpt2/)
    project_root = os.getcwd()

    # Get path to hpo_aware_pretrain
    hpo_aware_pretrain_dir = os.path.join(project_root, "models", "hpo_aware_pretrain")

    base_model_name = peft_config.base_model_name_or_path if os.path.isfile(peft_config.base_model_name_or_path) else hpo_aware_pretrain_dir
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )
    model = PeftModel.from_pretrained(model, model_id)
else: # either full finetuning or merged LoRA
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )
tokenizer_id = model_id
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id, use_fast = True)
model.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(146672, 4096, padding_idx=128256)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps

In [6]:
config = model.config
    if config.model_type == 'llama':
        tokenizer.chat_template = tokenizer.chat_template = """{% for message in messages %}
    {% if message['role'] == 'system' %}
    <|start_header_id|>system<|end_header_id|>
    {{ message['content'] }}<|eot_id|>
    {% elif message['role'] == 'user' %}
    <|start_header_id|>user<|end_header_id|>
    {{ message['content'] }}<|eot_id|>
    {% elif message['role'] == 'assistant' %}
    <|start_header_id|>assistant<|end_header_id|>
    {{ message['content'] }}<|eot_id|>
    {% endif %}
    {% endfor %}
    {% if add_generation_prompt %}
    <|start_header_id|>assistant<|end_header_id|>
    {% endif %}
    """

In [7]:
output_dir = 'example_testing/llama-arcus-split'
#output_dir = 'example_testing/qwen-arcus-split'
#output_dir = "/home/nguyenqm/projects/github/PhenoGPT2/phenogpt2_EHR_L318B_text_FPFF/evaluation/ID-68_deterministic/"
#output_dir = '/home/nguyenqm/projects/github/PhenoGPT2/phenogpt2_short_qwen3_4b/mimic_iv/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok = True)
print(output_dir)

example_testing/llama-arcus-split


In [8]:
emb_model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")

In [9]:
all_responses = {}
seen = 0
for index, dt in tqdm(data_input.items()):
    all_responses[index] = {}
    if use_text:
        text = data_input[index]['clinical_note'].lower()
        if wc != 0:
            all_chunks = chunking_documents(text, bert_tokenizer, bert_model, word_count = wc)
        else:
            all_chunks = [text]
        temp_response = {}
        for para_id, chunk in enumerate(all_chunks):
            chunk = chunk.replace("'", ""). replace('"', '')
            if len(all_chunks) > 1:
                pred_label = predict_label(bert_tokenizer, bert_model, {"text":chunk})
            else: # in case users only want to use the whole note for testing
                pred_label = 'INFORMATIVE'
            if pred_label == 'INFORMATIVE':
                # Try first attempt
                response = generate_output(model, tokenizer, chunk, temperature = 0.4, max_new_tokens = 3000, device = device)
                try:
                    final_response, complete_check = valid_json(response)
                    phenos = final_response.get("phenotypes", {})
                    if not isinstance(phenos, dict) or len(phenos) == 0:
                        raise ValueError("Empty or invalid phenotype dict")
                except Exception:
                    try:
                        response = generate_output(model, tokenizer, chunk, temperature=0.6, max_new_tokens=5000, device = device)
                        final_response, complete_check = valid_json(response)
                        phenos = final_response.get("phenotypes", {})
                        if not isinstance(phenos, dict) or len(phenos) == 0:
                            raise ValueError("Empty or invalid phenotype dict after retry")
                    except Exception as e:
                        print(f"Error: {e}", flush = True)
                        final_response = {'error_response': response}
                        final_response['pid'] = data_input[index].get('pid', data_input[index].get('pmid', 'unknown'))
                        temp_response[para_id] = final_response
                        continue  # move to the next item
                if negation:
                    print('Starting detecting negation')
                    try:
                        negation_response = negation_detection(negation_model, negation_tokenizer, chunk, final_response, device = device, max_new_tokens = 10000)
                        final_response = process_negation(final_response, negation_response, complete_check, emb_model)
                    except:
                        final_response['filtered_phenotypes'] = {}
                else:
                    final_response['filtered_phenotypes'] = {}
                # if seen <= 10: ## You can comment this out for logging some early results
                #     if len(final_response['filtered_phenotypes']) > 0:
                #         print(final_response['filtered_phenotypes'], flush = True)
                #         print(final_response['negation_analysis'], flush = True)
                #     else:
                #         print(final_response['negation_analysis'], flush = True)
                temp_response[para_id] = final_response
        if len(temp_response) > 1: # if splitting notes into multiple chunks, now merge all
            all_responses[index]['text'] = merge_outputs(temp_response)
        else:
            all_responses[index]['text'] = temp_response[0] # use the whole note as input
    else:
        all_responses[index]['text'] = {}
    seen += 1
    if use_vision:
        vision_phenotypes = phenogpt2_vision.generate_descriptions(dt['image'])
        phen2hpo = generate_output(model, tokenizer, vision_phenotypes, temperature = 0.4, max_new_tokens = 1024, device = device)
        phen2hpo = "{'demographics': {'age': '" + phen2hpo
        phen2hpo = valid_json(phen2hpo)
        phen2hpo = phen2hpo.get("phenotypes", {})
        try:
            phen2hpo = {phen:hpo_dict['HPO_ID'] for phen,hpo_dict in phen2hpo.items()}
        except:
            phen2hpo = {}
        all_responses[index]['image'] = phen2hpo
    else:
        all_responses[index]['image'] = {}

  0%|          | 0/854 [00:00<?, ?it/s]

Error: Empty or invalid phenotype dict after retry
Starting detecting negation
{'demographics': {'age': '8 years old', 'sex': 'female', 'ethnicity': 'unknown'}, 'phenotypes': {'global developmental delay': {'HPO_ID': 'HP:0001263', 'onset': 'unknown'}, 'autism': {'HPO_ID': 'HP:0000717', 'onset': 'unknown'}, 'short stature': {'HPO_ID': 'HP:0004322', 'onset': 'unknown'}, 'esotropia': {'HPO_ID': 'HP:0000565', 'onset': 'unknown'}, 'rop': {'HPO_ID': 'HP:0500049', 'onset': 'unknown'}, 'chronic kidney disease': {'HPO_ID': 'HP:0012622', 'onset': 'unknown'}, 'chronic lung disease': {'HPO_ID': 'HP:0006528', 'onset': 'unknown'}, 'laryngomalacia': {'HPO_ID': 'HP:0001601', 'onset': 'unknown'}, 'mild to moderate bilateral sensorineural hearing loss': {'HPO_ID': 'HP:0008504', 'onset': 'unknown'}, 'cystic renal dysplasia': {'HPO_ID': 'HP:0000800', 'onset': 'unknown'}}, 'filtered_phenotypes': {}}
###=
###=
###=
###=
###=
###=
###=
###=
###=
###=
###=
###=
###=
###=
###=
###=
###=
###=
###=
###=
###=
###

Starting detecting negation
{'demographics': {'age': 'unknown', 'sex': 'unknown', 'ethnicity': 'unknown'}, 'phenotypes': {'porencephaly': {'HPO_ID': 'HP:0002132', 'onset': 'unknown'}, 'periventricular leukoencephalopathy': {'HPO_ID': 'HP:0006970', 'onset': 'unknown'}, 'lacunar infarcts': {'HPO_ID': 'HP:0032325', 'onset': 'unknown'}, 'microbleeds': {'HPO_ID': 'HP:0032416', 'onset': 'unknown'}, 'intracerebral calcifications': {'HPO_ID': 'HP:0430048', 'onset': 'unknown'}, 'hemiparesis': {'HPO_ID': 'HP:0001269', 'onset': 'unknown'}, 'seizures': {'HPO_ID': 'HP:0001250', 'onset': 'unknown'}, 'intellectual disability': {'HPO_ID': 'HP:0001249', 'onset': 'unknown'}, 'dystonia': {'HPO_ID': 'HP:0001332', 'onset': 'unknown'}, 'stroke': {'HPO_ID': 'HP:0001297', 'onset': 'unknown'}, 'migraine': {'HPO_ID': 'HP:0002076', 'onset': 'unknown'}, 'cognitive impairment': {'HPO_ID': 'HP:0100543', 'onset': 'unknown'}, 'cataracts': {'HPO_ID': 'HP:0000518', 'onset': 'unknown'}, 'retinal arterial tortuosity': {'

Starting detecting negation
{'demographics': {'age': 'unknown', 'sex': 'female', 'ethnicity': 'unknown'}, 'phenotypes': {'developmental and cognitive delays': {'HPO_ID': 'HP:0001263', 'onset': 'unknown'}, 'brain imaging': {'HPO_ID': 'HP:0410263', 'onset': 'unknown'}, 'mitral valve prolapse': {'HPO_ID': 'HP:0001634', 'onset': 'unknown'}, 'muscle cramps': {'HPO_ID': 'HP:0003394', 'onset': 'unknown'}}, 'filtered_phenotypes': {'developmental and cognitive delays': {'HPO_ID': 'HP:0001263', 'onset': 'unknown'}, 'brain imaging': {'HPO_ID': 'HP:0410263', 'onset': 'unknown'}, 'mitral valve prolapse': {'HPO_ID': 'HP:0001634', 'onset': 'unknown'}, 'muscle cramps': {'HPO_ID': 'HP:0003394', 'onset': 'unknown'}}, 'complete': False}
SOLUTION
{'demographics': {'age': {'evidence': 'unknown', 'correct': 'False'},'sex': {'evidence': 'female', 'correct': 'True'}, 'ethnicity': {'evidence': 'unknown', 'correct': 'False'}}, 'phenotypes': {'developmental and cognitive delays': {'evidence': 'is known to have d


Merging all chunks together: 6it [00:00, 33870.56it/s]
  0%|          | 1/854 [12:51<182:41:05, 771.00s/it]

...

Starting detecting negation
Starting detecting negation
Starting detecting negation
Starting detecting negation
Starting detecting negation
Starting detecting negation
Error: Empty or invalid phenotype dict after retry
Error: Empty or invalid phenotype dict after retry
Error: Empty or invalid phenotype dict after retry
Starting detecting negation



Merging all chunks together: 10it [00:00, 55775.32it/s]
  3%|▎         | 23/854 [3:10:34<93:39:41, 405.75s/it] 

Starting detecting negation
Error: Empty or invalid phenotype dict after retry
Error: Empty or invalid phenotype dict after retry
Starting detecting negation
Starting detecting negation
Starting detecting negation
Starting detecting negation
Starting detecting negation
Starting detecting negation
Error: Empty or invalid phenotype dict after retry
Starting detecting negation
Error: Empty or invalid phenotype dict after retry
Error: Empty or invalid phenotype dict after retry
Starting detecting negation
Error: Empty or invalid phenotype dict after retry
Starting detecting negation



Merging all chunks together: 3it [00:00, 21732.15it/s]
  3%|▎         | 23/854 [3:20:56<121:00:05, 524.19s/it]


AttributeError: 'int' object has no attribute 'lower'

In [ ]:
with open(f'{output_dir}/phenogpt2.pkl', 'wb') as f:
    pickle.dump(all_responses, f)